In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
data = pd.read_csv(r'data/alquiler_AMBA_dev.csv')
print(data.shape)

C:\Users\03tom\AppData\Local\Temp\ipykernel_1580\3755051595.py:1: DtypeWarning: Columns (8,33,36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'data/alquiler_AMBA_dev.csv')


(278725, 45)


In [41]:
# Remove leading and trailing whitespace in column: 'BusinessCenter'
data['BusinessCenter'] = data['BusinessCenter'].str.replace("Sí", "1", case=False, regex=False)
data['BusinessCenter'] = data['BusinessCenter'].str.replace("No", "0", case=False, regex=False)
data['BusinessCenter'] = data['BusinessCenter'].str.replace("0.0", "0", case=False, regex=False)
data['BusinessCenter'] = data['BusinessCenter'].str.replace("1.0", "1", case=False, regex=False)
data['BusinessCenter'] = data['BusinessCenter'].str.strip()
print(data['BusinessCenter'].unique())

['0' nan '1']


## Preprocesamiento

In [35]:
from sklearn.model_selection import train_test_split

print(data['precio_pesos_constantes'].value_counts())
data_dev, data_test = train_test_split(data, test_size=5000/278725, random_state=42)
data_train, data_val = train_test_split(data_dev, test_size=0.2, random_state=42)

print(data_train.shape)
print(data_val.shape)
print(data_test.shape)

precio_pesos_constantes
7465.744761      779
7213.279962      776
6969.352620      742
7932.596479      724
7355.335072      723
                ... 
112761.114439      1
3217.882697        1
71851.947642       1
24410.067814       1
42798.794443       1
Name: count, Length: 14389, dtype: int64
(218980, 46)
(54745, 46)
(5000, 46)
